In [36]:
import os
import requests
import pandas as pd
from langchain_community.document_loaders.pdf import PyPDFDirectoryLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_community.embeddings import OpenAIEmbeddings
from langchain_community.llms import OpenAI
from langchain_community.vectorstores import Pinecone
from langchain.chains  import RetrievalQA
from langchain.prompts import PromptTemplate
from pinecone import Pinecone, ServerlessSpec
from dotenv import load_dotenv


ImportError: cannot import name 'from_texts' from 'pinecone' (/Users/loke/Library/CloudStorage/Dropbox/Project/LLM Projects/LLM_MCQ_Generator/llmmcq/lib/python3.9/site-packages/pinecone/__init__.py)

In [5]:
pdf_dir = PyPDFDirectoryLoader('pdfs')
data = pdf_dir.load()

In [7]:
data_splitter = RecursiveCharacterTextSplitter(chunk_size = 500,
                               chunk_overlap = 20
                               )

data_chunks = data_splitter.split_documents(data)
data_chunks


[Document(page_content='Time Series Moving ARIMA  \n \nBusiness Objective  \n \nA time series is simply a series of data points ordered in time. In a time -series, time is \noften the independent variable, and the goal is usually to make a forecast for the future.  \n \nTime series data can be helpful for many applications in day -to-day activities like:   \n\uf0b7 Tracking daily, hourly, or weekly weather data  \n\uf0b7 Monitoring changes in application performance  \n\uf0b7 Medical devices to visualize vitals in real -time', metadata={'source': 'pdfs/Solution methodology.pdf', 'page': 0}),
 Document(page_content='Auto-Regressive Integrated Moving Average (ARIMA) m odel is one of the more \npopular and widely used statistical methods for time -series forecasting. ARIMA is an \nacronym that stands for Auto -Regressive Integrated Moving Average. It is a class of \nstatistical algorithms that captures the standard temporal depe ndencies unique to \ntime-series data.  \n \nThe model is us

In [10]:
print(data_chunks[0].page_content)

Time Series Moving ARIMA  
 
Business Objective  
 
A time series is simply a series of data points ordered in time. In a time -series, time is 
often the independent variable, and the goal is usually to make a forecast for the future.  
 
Time series data can be helpful for many applications in day -to-day activities like:   
 Tracking daily, hourly, or weekly weather data  
 Monitoring changes in application performance  
 Medical devices to visualize vitals in real -time


In [25]:
load_dotenv()

model_id = "sentence-transformers/all-MiniLM-L6-v2"
hf_token = os.getenv('HF_TOKEN')

In [27]:

api_url = f"https://api-inference.huggingface.co/pipeline/feature-extraction/{model_id}"
headers = {"Authorization": f"Bearer {hf_token}"}

def query(texts):
    response = requests.post(api_url, headers=headers, json={"inputs": texts, "options":{"wait_for_model":True}})
    return response.json()

In [28]:
texts = ["How do I get a replacement Medicare card?",
        "What is the monthly premium for Medicare Part B?",
        "How do I terminate my Medicare Part B (medical insurance)?",
        "How do I sign up for Medicare?",
        "Can I sign up for Medicare Part B if I am working and have health insurance through an employer?",
        "How do I sign up for Medicare Part B if I already have Part A?",
        "What are Medicare late enrollment penalties?",
        "What is Medicare and who can get it?",
        "How can I get help with my Medicare Part A and Part B premiums?",
        "What are the different parts of Medicare?",
        "Will my Medicare premiums be higher because of my higher income?",
        "What is TRICARE ?",
        "Should I sign up for Medicare Part B if I have Veterans' Benefits?"]

output = query(texts)
embeddings =pd.DataFrame(output)

In [46]:
embeddings_df = pd.DataFrame({
    'id': texts,
    'vector': output
})

In [47]:
embeddings_df

,id,vector
0,How do I get a replacement Medicare card?,"[-0.023889480158686638, 0.055258527398109436, ..."
1,What is the monthly premium for Medicare Part B?,"[-0.012687616981565952, 0.046874117106199265, ..."
2,How do I terminate my Medicare Part B (medical...,"[0.0004943753010593355, 0.11941202729940414, 0..."
3,How do I sign up for Medicare?,"[-0.029711339622735977, 0.023298313841223717, ..."
4,Can I sign up for Medicare Part B if I am work...,"[-0.02562847174704075, 0.07038860023021698, -0..."
5,How do I sign up for Medicare Part B if I alre...,"[-0.022656138986349106, 0.021159760653972626, ..."
6,What are Medicare late enrollment penalties?,"[-0.002911375602707267, 0.060791268944740295, ..."
7,What is Medicare and who can get it?,"[-0.08052559196949005, 0.05988813564181328, -0..."
8,How can I get help with my Medicare Part A and...,"[-0.034387730062007904, 0.07250141352415085, 0..."
9,What are the different parts of Medicare?,"[-0.005963803268969059, 0.025043729692697525, ..."


In [48]:
load_dotenv()
api_key = os.getenv('PINECONE_API_KEY')
pc = Pinecone(api_key= api_key)
index_name = 'testing'
index = pc.Index(index_name)
index.upsert(vectors=zip(embeddings_df.id,embeddings_df.vector))

{'upserted_count': 13}